In [ ]:
# ============================================================================
# SERIES TEMPORALES - EJEMPLO SIMPLE Y DINÁMICO
# CASO: Predecir cuándo un estudiante ABANDONA una clase online
# ============================================================================

# PASO 1: IMPORTAR LIBRERÍAS NECESARIAS
# Explicación: pandas para tablas, numpy para números, matplotlib para gráficos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print("="*70)
print("SERIES TEMPORALES - PREDICCIÓN DE ABANDONO EN CLASES ONLINE")
print("="*70)
print("\nCASO: Un estudiante se inscribe en una clase online.")
print("¿Cuándo la ABANDONA?")
print("¿Podemos PREDECIRLO ANTES de que pase?\n")

# ============================================================================
# PASO 2: CREAR DATOS DE UN ESTUDIANTE EN 60 DÍAS
# ============================================================================

# Crear fechas: desde hoy hace 60 días
fecha_inicio = datetime(2024, 1, 1)
# Generar lista con cada día (rango de 60 días)
fechas = [fecha_inicio + timedelta(days=i) for i in range(60)]

# Crear el EVENTO que mediremos: TIEMPO DE CONEXIÓN (en minutos)
# Explicación: Cada día el estudiante se conecta X minutos
# Si se conecta poco/nada durante días = probable abandono

tiempo_conexion = []

for i in range(60):
    # COMPONENTE 1: TENDENCIA
    # El estudiante PIERDE INTERÉS con el tiempo
    # Semana 1: 45 min/día | Semana 8: 5 min/día
    tendencia = 50 - (i * 0.7)  # Disminuye 0.7 minutos cada día
    
    # COMPONENTE 2: ESTACIONALIDAD SEMANAL
    # Patrón: Más tiempo entre semana (estudia), menos fin de semana
    dia_semana = i % 7  # 0=lunes, 6=domingo
    if dia_semana in [0, 1, 2, 3, 4]:  # Lunes a viernes
        bonus_semana = 15  # Le agrega 15 min extra
    else:  # Sábado, domingo
        bonus_semana = -10  # Pierde 10 min
    
    # COMPONENTE 3: RUIDO (ALEATORIO)
    # A veces tiene tareas extra, a veces está ocupado
    ruido = np.random.normal(0, 5)  # Variación aleatoria de ±5 min
    
    # CALCULAR TOTAL
    minutos_dia = tendencia + bonus_semana + ruido
    # Asegurar que no sea negativo
    minutos_dia = max(0, minutos_dia)
    
    tiempo_conexion.append(minutos_dia)

# Crear tabla (DataFrame) con estos datos
df = pd.DataFrame({
    'fecha': fechas,
    'minutos_conectado': tiempo_conexion
})

# Establecer la fecha como índice (importante para series temporales)
df.set_index('fecha', inplace=True)

print("[DATOS GENERADOS]")
print(f"- Período: {df.index.min().date()} a {df.index.max().date()}")
print(f"- Total de días: {len(df)}")
print(f"\nPrimeros 5 días:")
print(df.head(5).round(1))
print(f"\nÚltimos 5 días:")
print(df.tail(5).round(1))

# ============================================================================
# PASO 3: GRAFICAR LA SERIE TEMPORAL COMPLETA
# ============================================================================

print("\n" + "="*70)
print("GRÁFICO 1: LA SERIE TEMPORAL COMPLETA")
print("="*70)

# Crear figura (lienzo para dibujar)
fig, ax = plt.subplots(figsize=(12, 6))

# Plotear línea (representar datos en gráfico)
ax.plot(df.index, df['minutos_conectado'], linewidth=2.5, color='#2563eb', marker='o', markersize=4)

# Llenar área bajo la línea (hacer más visible)
ax.fill_between(df.index, df['minutos_conectado'], alpha=0.2, color='#2563eb')

# Línea de alerta: si baja de 10 minutos por día = abandono probable
ax.axhline(y=10, color='red', linestyle='--', linewidth=2, label='Alerta: 10 min (abandono probable)')

# Etiquetas y título
ax.set_title('Tiempo de Conexión Diaria de un Estudiante (60 días)', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Minutos Conectado', fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("Interpretación: El estudiante ESTÁ CONECTADO CADA VEZ MENOS")
print("Empieza con 45 min, termina con solo 5 min")

# ============================================================================
# PASO 4: IDENTIFICAR COMPONENTES
# ============================================================================

print("\n" + "="*70)
print("COMPONENTES DE LA SERIE TEMPORAL")
print("="*70)

# TENDENCIA: Usar media móvil de 7 días
# Explicación: Promedio de 7 días consecutivos
# Esto "suaviza" datos y muestra dirección general
tendencia = df['minutos_conectado'].rolling(window=7, center=True).mean()

# ESTACIONALIDAD: Lo que queda después de restar la tendencia
# Explicación: Son los picos y valles semanales
estacionalidad = df['minutos_conectado'] - tendencia

print("\n1. TENDENCIA (Dirección general)")
print(f"   - Inicio: {tendencia.iloc[10]:.1f} minutos")
print(f"   - Final: {tendencia.iloc[-10]:.1f} minutos")
print(f"   - Conclusión: BAJA consistente (pierde interés)")

print("\n2. ESTACIONALIDAD (Patrón semanal)")
print(f"   - Entre semana: Más conexión")
print(f"   - Fin de semana: Menos conexión")
print(f"   - Conclusión: Estudia cuando tiene obligaciones")

# ============================================================================
# PASO 5: GRAFICAR COMPONENTES
# ============================================================================

print("\n" + "="*70)
print("GRÁFICO 2: DESCOMPOSICIÓN EN COMPONENTES")
print("="*70)

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Gráfico 1: Original vs Tendencia
axes[0].plot(df.index, df['minutos_conectado'], linewidth=1, color='lightblue', alpha=0.7, label='Datos reales')
axes[0].plot(df.index, tendencia, linewidth=3, color='darkred', label='Tendencia (dirección)')
axes[0].set_title('COMPONENTE 1: TENDENCIA', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Minutos', fontsize=11)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Estacionalidad
axes[1].plot(df.index, estacionalidad, linewidth=2, color='darkgreen', marker='s', markersize=3)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].fill_between(df.index, estacionalidad, 0, alpha=0.3, color='green')
axes[1].set_title('COMPONENTE 2: ESTACIONALIDAD (Patrón semanal)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Desviación (minutos)', fontsize=11)
axes[1].set_xlabel('Fecha', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("La tendencia baja = está perdiendo interés cada vez más")

# ============================================================================
# PASO 6: CALCULAR INDICADOR DE ABANDONO
# ============================================================================

print("\n" + "="*70)
print("INDICADOR CRÍTICO: RIESGO DE ABANDONO")
print("="*70)

# Crear columna: días consecutivos con poco tiempo (<10 minutos)
# Explicación: Contar cuántos días seguidos estudia muy poco
dias_baja_conexion = 0
contador_dias = []

for minutos in df['minutos_conectado']:
    if minutos < 10:  # Si estudia menos de 10 minutos
        dias_baja_conexion += 1  # Contar un día más
    else:
        dias_baja_conexion = 0  # Si estudia bien, reiniciar contador
    
    contador_dias.append(dias_baja_conexion)

# Agregar esta columna a la tabla
df['dias_bajo_uso'] = contador_dias

# Crear variable de ALERTA
# Si tiene 5+ días seguidos con bajo uso = ABANDONO PROBABLE
df['en_riesgo'] = df['dias_bajo_uso'] >= 5

print(f"\nEstadísticas:")
print(f"- Máximo días consecutivos sin estudiar: {df['dias_bajo_uso'].max()}")
print(f"- Días en RIESGO (5+): {df['en_riesgo'].sum()}")

if df['en_riesgo'].sum() > 10:
    print("\n⚠️ RESULTADO: Este estudiante se ABANDONA la clase")
    print("   (Tiene demasiados días con muy poca actividad)")
else:
    print("\n✓ RESULTADO: Este estudiante permanece en la clase")

# ============================================================================
# PASO 7: GRAFICAR INDICADOR DE ABANDONO
# ============================================================================

print("\n" + "="*70)
print("GRÁFICO 3: INDICADOR DE ABANDONO")
print("="*70)

fig, ax = plt.subplots(figsize=(12, 6))

# Colorear según riesgo: rojo si en riesgo, verde si no
colores = ['red' if x else 'green' for x in df['en_riesgo']]

# Graficar barras con colores
ax.bar(df.index, df['minutos_conectado'], color=colores, alpha=0.6, edgecolor='black')

# Línea de alerta
ax.axhline(y=10, color='orange', linestyle='--', linewidth=2, label='Umbral de alerta (10 min)')

# Etiquetas
ax.set_title('Indicador de Abandono: Tiempo de Conexión', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Minutos Conectado', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Leyenda personalizada
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='green', alpha=0.6, label='Conexión normal'),
                   Patch(facecolor='red', alpha=0.6, label='Riesgo de abandono')]
ax.legend(handles=legend_elements, fontsize=11)

plt.tight_layout()
plt.show()

# ============================================================================
# PASO 8: CONCLUSIÓN Y APLICACIÓN
# ============================================================================

print("\n" + "="*70)
print("CONCLUSIÓN: ¿QUÉ ES SERIES TEMPORALES?")
print("="*70)

conclusion = """
SIN Series Temporales:
  "El estudiante tiene 2000 minutos de conexión total en 60 días"
  (Es un número, pero no nos dice NADA importante)

CON Series Temporales:
  "El estudiante EMPEZÓ con 45 min/día
   Ahora solo se conecta 5 min/día
   Tiene 15 días seguidos sin estudiar
   Se va a ABANDONAR EN LOS PRÓXIMOS 3 DÍAS
   Recomendación: Enviarle motivación/ayuda AHORA"

DIFERENCIA: Series Temporales responde:
  ¿QUÉ ESTÁ PASANDO? ¿CUÁNDO PASARÁ? ¿POR QUÉ?

APLICACIONES EN LA VIDA REAL:
  - Streaming: Predecir quién se cancela (como Netflix)
  - Redes sociales: Detectar usuarios inactivos
  - Banco: Detectar fraude (transacciones anormales)
  - E-commerce: Predecir fin de temporada
  - Salud: Monitorear pacientes en el tiempo
"""

print(conclusion)

print("\n" + "="*70)
print("FIN - Ya entiendes Series Temporales")
print("="*70)